# Notebook 1 — Data Loading & Initial Cleaning  
This notebook loads the raw SCADA data, performs early inspection, renames key columns, converts data to numeric formats, and removes rows with missing values in the essential variables.

**Goals of this notebook:**
- Load the raw SCADA `.csv` file  
- Inspect the dataset shape and preview a sample  
- Standardize column names  
- Convert important columns to numeric  
- Check for missing values  
- Drop incomplete rows  
- Prepare a clean dataset for *Region 3 filtering* in the next notebook


# 📥 1. Load the Dataset

In [1]:
import pandas as pd

# Try loading from one directory up
df = pd.read_csv("../data/raw_scada.csv", skiprows=9)

# Quick check
print(df.shape)
df.head()


(26064, 299)


,# Date and time,Wind speed (m/s),"Wind speed, Standard deviation (m/s)","Wind speed, Minimum (m/s)","Wind speed, Maximum (m/s)",Long Term Wind (m/s),Wind speed Sensor 1 (m/s),"Wind speed Sensor 1, Standard deviation (m/s)","Wind speed Sensor 1, Minimum (m/s)","Wind speed Sensor 1, Maximum (m/s)",...,Tower Acceleration y (mm/ss),"Tower Acceleration X, Min (mm/ss)","Tower Acceleration X, Max (mm/ss)","Tower Acceleration Y, Min (mm/ss)","Tower Acceleration Y, Max (mm/ss)","Drive train acceleration, Max (mm/ss)","Drive train acceleration, Min (mm/ss)","Drive train acceleration, StdDev (mm/ss)","Tower Acceleration X, StdDev (mm/ss)","Tower Acceleration Y, StdDev (mm/ss)"
0,2021-01-01 00:00:00,4.038019,0.700345,2.551292,5.217981,7.1,4.126690,0.563626,3.120180,5.439071,...,38.765776,29.741989,274.860626,6.783698,68.157379,3.980548,3.980548,0.0,59.474656,17.885789
1,2021-01-01 00:10:00,3.710743,0.583452,2.508970,5.101780,7.1,3.986455,0.567087,2.935361,4.973041,...,29.165829,25.464008,253.534164,7.835090,58.152035,4.652066,4.652066,0.0,53.743395,13.314676
2,2021-01-01 00:20:00,4.118079,0.713885,2.725780,5.331213,7.1,4.165041,0.582563,3.219593,5.317415,...,29.821561,22.918325,207.772491,6.318348,84.116875,5.305034,5.305034,0.0,46.453577,20.196765
3,2021-01-01 00:30:00,4.365870,0.688869,3.039486,5.903680,7.1,4.576696,0.687357,3.147917,5.626087,...,33.539071,39.921917,203.347443,6.269512,63.643074,4.835838,4.835838,0.0,44.439622,16.916373
4,2021-01-01 00:40:00,4.376747,0.737255,2.845323,5.549507,7.1,4.402202,0.667562,2.998249,5.481088,...,25.737522,11.115573,154.934570,9.203635,47.022877,4.706601,4.706601,0.0,37.945509,11.704536


# 🔍 2. Inspect Important Columns

In [16]:
# Preview key columns
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].head(10))

# See null counts
print("\nMissing values per column:")
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].isna().sum())

# See data types
print("\nData types:")
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].dtypes)


Empty DataFrame
Columns: [wind_speed, rotor_speed, power, pitch]
Index: []

Missing values per column:
wind_speed     0
rotor_speed    0
power          0
pitch          0
dtype: int64

Data types:
wind_speed     float64
rotor_speed    float64
power          float64
pitch          float64
dtype: object


# 🛠️ 3. Rename Columns & Convert to Numeric

In [18]:
# Rename important columns if needed (adjust these names based on what your CSV has)
df.rename(columns={
    "Wind speed (m/s)": "wind_speed",
    "Rotor speed (RPM)": "rotor_speed",
    "Power (kW)": "power",
    "Blade angle (pitch position) A (°)": "pitch"
}, inplace=True)

# Convert to numeric (will coerce non-numeric values to NaN)
df["wind_speed"] = pd.to_numeric(df["wind_speed"], errors="coerce")
df["rotor_speed"] = pd.to_numeric(df["rotor_speed"], errors="coerce")
df["power"] = pd.to_numeric(df["power"], errors="coerce")
df["pitch"] = pd.to_numeric(df["pitch"], errors="coerce")

# Show updated nulls and dtypes after conversion
print("\nPost-cleaning missing values:")
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].isna().sum())

print("\nPost-cleaning data types:")
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].dtypes)

# Drop rows with missing values in any of the four key columns
df.dropna(subset=["wind_speed", "rotor_speed", "power", "pitch"], inplace=True)

# Confirm again
print("\nShape after dropping missing values:", df.shape)
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].head(5))



Post-cleaning missing values:
wind_speed     0
rotor_speed    0
power          0
pitch          0
dtype: int64

Post-cleaning data types:
wind_speed     float64
rotor_speed    float64
power          float64
pitch          float64
dtype: object

Shape after dropping missing values: (0, 300)
Empty DataFrame
Columns: [wind_speed, rotor_speed, power, pitch]
Index: []


In [19]:
# Reload the dataset (clean slate)
df = pd.read_csv("../data/raw_scada.csv", skiprows=9)

# Rename key columns
df.rename(columns={
    "Wind speed (m/s)": "wind_speed",
    "Rotor speed (RPM)": "rotor_speed",
    "Power (kW)": "power",
    "Blade angle (pitch position) A (°)": "pitch"
}, inplace=True)

# Convert and inspect BEFORE dropping
for col in ["wind_speed", "rotor_speed", "power", "pitch"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert invalids to NaN

# 👁️ See how many rows will be dropped
print("\nMissing per column before drop:")
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].isna().sum())
print("\nTotal rows before dropping:", df.shape)

# Drop rows where any of the 4 columns is NaN
df.dropna(subset=["wind_speed", "rotor_speed", "power", "pitch"], inplace=True)

# Confirm again
print("\nAfter dropping rows with missing values:")
print("Shape:", df.shape)
print(df[["wind_speed", "rotor_speed", "power", "pitch"]].head(5))



Missing per column before drop:
wind_speed       68
rotor_speed      68
power            68
pitch          2250
dtype: int64

Total rows before dropping: (26064, 299)

After dropping rows with missing values:
Shape: (23814, 299)
   wind_speed  rotor_speed       power   pitch
0    4.038019     9.502076  176.543091  0.4195
1    3.710743     9.559778  119.758620  0.7900
2    4.118079     9.522331  142.842966  0.7080
3    4.365870     9.431067  181.747184  0.2700
4    4.376747     9.428509  178.464342  0.3695


In [20]:
# Filter Region 3
region3 = df[(df["wind_speed"] >= 10.0) & (df["rotor_speed"] >= 9.0)]

# Confirm shape and preview
print("Region 3 shape:", region3.shape)
print(region3[["wind_speed", "rotor_speed", "power", "pitch"]].head())


Region 3 shape: (1488, 299)
      wind_speed  rotor_speed        power     pitch
1476   10.236168    15.160395  1895.012958  0.888500
1489   10.186494    15.130147  1869.430042  1.239500
1499   10.226459    15.167897  1921.307123  2.567500
1500   10.452997    15.205657  2001.144473  3.696842
1509   10.478650    15.170470  1979.384882  3.739000


In [21]:
# Features and target
X = region3[["wind_speed", "rotor_speed", "power"]]
y = region3["pitch"]


# 📦 5. Save Cleaned Dataset

In [28]:
# Save prepared Region 3 dataset
region3.to_csv("../data/region3_clean.csv", index=False)
print("✅ Region 3 cleaned dataset saved successfully.")


✅ Region 3 cleaned dataset saved successfully.
